In [1]:
from calculate import calculatePHI_LIQUID_NONESRK
from calculate import calculatePHI_VAPOR_SRK
from read import read_excel,getTHETA
from calcute_theta import predict_THETA
import numpy as np
import matplotlib.pyplot as plt
from math import *

In [2]:
features = [
    'Tr', 'Pr', 'LIQUID PL', 'LIQUID K', 'LIQUID MU', 'LIQUID SIGMA',
    'TOTAL RHO', 'LIQUID DHVL', 'M', 'Tc', 'Pc',
    'FREEZEPT', 'TB', 'omiga'
]

In [3]:
pres = read_excel('./DATA/物性系数.xlsx',0,1)
# we test 110-62-3
TEST_LIST = pres['107-92-6']
SATlist = TEST_LIST[9:18] # 计算饱和蒸气压经验公式系数

Matterdata = read_excel('./DATA/test.xlsx',1,4)
KEY_SET = list(Matterdata.keys())
TEMP = Matterdata[KEY_SET[0]][1:] # K
PESS = Matterdata[KEY_SET[1]][1:] # Pa
VOLE = list(np.array(Matterdata[KEY_SET[2]][1:])/(10**6)) # mL/mol
TC = Matterdata[KEY_SET[10]][1:] # K
PC = Matterdata[KEY_SET[11]][1:] # Pa
OMEGA = Matterdata[KEY_SET[16]][1:] # 1
M = Matterdata[KEY_SET[9]][1:] # g/mol
THETALIST = []
for i in range(len(TEMP)):
	thetat = getTHETA(PESS[i],VOLE[i],TEMP[i],TC[i],PC[i],OMEGA[i])
	THETALIST.append(thetat)
TR = list(np.array(TEMP)/np.array(TC))
PR = list(np.array(PESS)/np.array(PC))
DATASET = {
	'Tr':TR,
	'pr':PR,
	KEY_SET[3]:Matterdata[KEY_SET[3]][1:],
	KEY_SET[4]:Matterdata[KEY_SET[4]][1:],
	KEY_SET[5]:Matterdata[KEY_SET[5]][1:],
	KEY_SET[6]:Matterdata[KEY_SET[6]][1:],
	KEY_SET[7]:Matterdata[KEY_SET[7]][1:],
	KEY_SET[8]:Matterdata[KEY_SET[8]][1:],
	'M(g/mol)':M,
	'Tc(K)':TC,
	'pc(Pa)':PC,
	KEY_SET[14]:Matterdata[KEY_SET[14]][1:],
	KEY_SET[15]:Matterdata[KEY_SET[15]][1:],
	'w':OMEGA,
}
print(THETALIST)

[np.float64(0.7718733997141937), np.float64(0.7723392164348405), np.float64(0.7728005905820732), np.float64(0.7732575976199663), np.float64(0.773710311091517), np.float64(0.7741588026845733), np.float64(0.7740535836200232), np.float64(0.7746472467665385), np.float64(0.7752344672484943), np.float64(0.775815371952685), np.float64(0.7763900839932422), np.float64(0.7769587228631092), np.float64(0.7753807094414149), np.float64(0.7761423433624118), np.float64(0.7768944613965344), np.float64(0.7776372836214601), np.float64(0.778371022379592), np.float64(0.7790958826455562), np.float64(0.7756325816173478), np.float64(0.7766192084682014), np.float64(0.7775914340311563), np.float64(0.7785496555995731), np.float64(0.7794942537139237), np.float64(0.7804255931167114), np.float64(0.774497659011782), np.float64(0.7757933278306671), np.float64(0.7770664816375185), np.float64(0.7783178761121973), np.float64(0.7795482279929644), np.float64(0.7807582177859027), np.float64(0.7715215548067377), np.float64(

In [4]:
thetacc = []
KEY_SET2 = list(DATASET.values())
for i in range(len(THETALIST)):
	feature = [c[i] for c in KEY_SET2]
	input_feature = {features[j]:feature[j] for j in range(len(feature))}
	theta_pre = predict_THETA(input_feature)
	thetacc.append(theta_pre)
print(thetacc)
print(input_feature)

[np.float64(0.8391690825182496), np.float64(0.839169076791098), np.float64(0.8391690710639463), np.float64(0.8391690653367949), np.float64(0.8391690596096433), np.float64(0.8391690538824916), np.float64(0.8320637148416876), np.float64(0.8320637101502291), np.float64(0.8320637054587701), np.float64(0.8320637007673117), np.float64(0.8320636960758527), np.float64(0.832063691384394), np.float64(1.03259347430041), np.float64(1.032593469808473), np.float64(1.0325934653165365), np.float64(1.0325934608245997), np.float64(1.032593456332663), np.float64(1.0325934518407258), np.float64(1.7161001030388494), np.float64(1.7161000992725037), np.float64(1.7161000955061583), np.float64(1.7161000917398124), np.float64(1.7161000879734665), np.float64(1.7161000842071206), np.float64(3.036136516621555), np.float64(3.0361365126557094), np.float64(3.036136508689863), np.float64(3.0361365047240163), np.float64(3.0361365007581687), np.float64(3.0361364967923246), np.float64(5.266571158375448), np.float64(5.266

In [5]:
# 计算逸度
phi = []
for k in range(len(THETALIST)):
	Phi = calculatePHI_LIQUID_NONESRK(PR[k]*PC[k],VOLE[k],TR[k]*TC[k],TC[k],PC[k],OMEGA[k],THETALIST[k],SATlist)
	phi.append(Phi)
print(phi)

[0.021670172100212503, 0.010048707717372355, 0.006633299736141895, 0.005001858029651575, 0.004047126360862415, 0.0034211104887145447, 0.060412864019889025, 0.027993992568879664, 0.01846600478443584, 0.013914399399001201, 0.011250470974095948, 0.009503484931455468, 0.14664321115081672, 0.06791066499717938, 0.044770109777027375, 0.033715000390322936, 0.027244212075494424, 0.023000251604272912, 0.3169331259853501, 0.14670080754434459, 0.0966659443035312, 0.07276142565941955, 0.0587687918597613, 0.04959081908542233, 0.6207754638575174, 0.28723255677851167, 0.18919585757434396, 0.1423570086739165, 0.11493872213512397, 0.09695386292391683, 1.1178651498408698, 0.5170912381437169, 0.3405082872401722, 0.2561421809921843, 0.20675603401118198, 0.1743611364926697, 1.8728078543219795, 0.8661532095641838, 0.570275772696956, 0.42891721857229853, 0.3461710144657429, 0.2918951665464574, 1.363554003451617, 0.8977249255286855, 0.6751828293788805, 0.5449240831363856, 0.4594905796989924, 2.034655120058143,